#### Data Importing

In [1]:
import pandas as pd
import numpy as np
breaches = pd.read_csv("Balloon Race Data Breaches - LATEST - breaches.csv")
stocks = pd.read_csv("World-Stock-Prices-Dataset.csv")

#### Data Cleaning and Formatting for Merging
* Ensuring merging columns are all lower case
* Renaming merging columns to the same name - organisation
* Renaming date columns from both datasets to different names to prevent any issues
* Formatting date columns into datetime type and specifying into 'Month - Year' Format for future merging requirements
* Removing unnecessary columns

In [2]:
breaches['organisation'] = breaches['organisation'].str.lower() 
stocks['Brand_Name'] = stocks['Brand_Name'].str.lower() 

In [3]:
stocks = stocks.rename(columns={'Brand_Name': 'organisation'})
stocks = stocks.rename(columns={'Date': 'StockDate'})
breaches = breaches.rename(columns={'date': 'BreachDate'}) 

In [4]:
breaches['BreachDate'] = pd.to_datetime(breaches['BreachDate'], format='mixed', utc=True)
breaches['breach_month_year'] = breaches['BreachDate'].dt.strftime('%B %Y')

stocks['StockDate'] = pd.to_datetime(stocks['StockDate'], utc=True)
stocks['stock_month_year'] = stocks['StockDate'].dt.strftime('%B %Y')

In [5]:
breaches = breaches.drop(columns=["alternative name","story","interesting story","Unnamed: 11", "source name", "1st source link", "2nd source link", "ID"])
stocks = stocks.drop(columns=["Open","High","Low","Ticker","Dividends","Stock Splits","Capital Gains"])

#### Performing Monthly Stock Calculation
By extracting month from the date column, it is possible to group rows by month and organisation to preform a mean column calculation to create a new monthly average column which uses the Stock price at the end of the day 'Close'

In [6]:
stocks['year_monthstocks'] = stocks['StockDate'].dt.to_period('M')

monthly_average = []
stocks['monthly_average'] = monthly_average = stocks.groupby(['organisation','year_monthstocks'])['Close'].transform('mean')

C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_16680\1598989567.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stocks['year_monthstocks'] = stocks['StockDate'].dt.to_period('M')


#### Merging Datasets 
* Left Join on the organisation name
* Removing n/a values in rows
* Identifying which organisations we can analyse

In [7]:
merged_data_stocks = pd.merge(stocks, breaches, on='organisation', how='left')

merged_data_stocks = merged_data_stocks.dropna(subset=['organisation','records lost']) 

distinct_values = merged_data_stocks['organisation'].unique()
print(distinct_values)

['peloton' 'amazon' 'apple' 'target' 'spotify' 'starbucks' 'nvidia'
 'adobe' 'toyota' 'uber' 'microsoft' 'ubisoft' 'nintendo' 'twitter']


#### Cleaning Post Merge 
Renaming sectors and methods for readability

In [8]:
merged_data_stocks['sector'] = merged_data_stocks['sector'].replace('web', 'tech')
merged_data_stocks['sector'] = merged_data_stocks['sector'].replace('tech, web', 'tech')
merged_data_stocks['method'] = merged_data_stocks['method'].replace('oops!', 'human error')

#### Saving as CSV - large dataframe with all dates of stocks for analysis

In [15]:
df = pd.DataFrame(merged_data_stocks)
df.to_csv('merged_data_stocks_full2.csv', index=False) 
merged_data_stocks 

,organisation,Country,stock_month_year,monthly_average,records lost,year,sector,method,data sensitivity,breach_month_year
0,peloton,usa,July 2025,6.622500,"3,000,000",2021,tech,poor security,2,May 2021
3,amazon,usa,July 2025,221.800003,"5,000,000",2018,retail,human error,1,November 2018
4,apple,usa,July 2025,211.840004,"275,000",2013,tech,hacked,1,July 2013
6,target,usa,July 2025,104.354998,"70,000,000",2013,retail,hacked,3,December 2013
8,spotify,usa,July 2025,720.487488,"500,000",2020,app,human error,1,December 2020
...,...,...,...,...,...,...,...,...,...,...
318171,starbucks,usa,January 2000,2.593795,"97,000",2008,retail,lost device,2,November 2008
318178,amazon,usa,January 2000,3.402500,"5,000,000",2018,retail,human error,1,November 2018
318181,nintendo,japan,January 2000,3.893261,"300,000",2020,gaming,hacked,3,April 2020
318182,nintendo,japan,January 2000,3.893261,"4,000,000",2013,gaming,hacked,2,June 2013


#### Matching Breach Date to Monthly Avg Stock Price
* Removing columns which had distinct values to select distinct rows without repeats
* Matching breach date to stock date and dropping any duplicates

In [10]:
merged_data_stocks =merged_data_stocks.drop(columns=["StockDate","Close","Volume","BreachDate","year_monthstocks",'displayed records','Industry_Tag'])
matching_dates = merged_data_stocks[merged_data_stocks['breach_month_year'] == merged_data_stocks['stock_month_year']].drop_duplicates()

In [11]:
matching_dates = matching_dates.round(2)
matching_dates['records lost'] = matching_dates['records lost'].str.replace(',','')
matching_dates['records lost'] = matching_dates['records lost'].astype('int')

In [12]:
matching_dates.sort_values('records lost', ascending=False)

,organisation,Country,stock_month_year,monthly_average,records lost,year,sector,method,data sensitivity,breach_month_year
88249,microsoft,usa,January 2020,158.36,250000000,2020,tech,poor security,1,January 2020
169904,target,usa,December 2013,47.62,70000000,2013,retail,hacked,3,December 2013
175187,ubisoft,france,July 2013,2.94,58000000,2013,gaming,hacked,2,July 2013
171962,adobe,usa,October 2013,52.40,38000000,2013,tech,hacked,3,October 2013
104875,amazon,usa,November 2018,81.27,5000000,2018,retail,human error,1,November 2018
176308,nintendo,japan,June 2013,2.63,4000000,2013,gaming,hacked,2,June 2013
99414,toyota,japan,April 2019,122.63,3100000,2019,transport,hacked,2,April 2019
68131,peloton,usa,May 2021,96.05,3000000,2021,tech,poor security,2,May 2021
74432,spotify,usa,December 2020,324.56,500000,2020,app,human error,1,December 2020
84603,nintendo,japan,April 2020,10.47,300000,2020,gaming,hacked,3,April 2020


#### Saving as CSV - smaller dataframe for specific months of databreaches

In [13]:
df = pd.DataFrame(matching_dates)
df.to_csv('stocksmerge2.csv', index=False)